Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [ ]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
import model
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import utils
import loop
import pickle
import glob2
import os
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
category = "balanced_data"
path = "../"
args = {
    "dims": [499, 256, 64, 32],
    "highly_genes": 500,
    'alpha': 0.001,
    'gamma': 0.001,
    'learning_rate': 0.0001,
    'update_epoch': 10,
    't_alpha': 1,
    'error': 0.001
}
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d
args = objectview(args)

In [ ]:
os.makedirs(f"{path}output/{category}", exist_ok = True)
os.makedirs(f"{path}output/pickle_results/{category}", exist_ok = True)

In [ ]:
files = glob2.glob(f'../R/simulated_data/{category}/*.h5')
files = [f[len(f"{path}R/simulated_data/{category}/"):-3] for f in files]
files

In [ ]:
for run in range(3):
    df = pd.DataFrame()
    print(df.shape)
    for dataset in files:
        print(f">>>>> Data {dataset}")

        data_mat = h5py.File(f"{path}R/simulated_data/{category}/{dataset}.h5", "r")
        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        print(f"Dropout level {data_mat['dropout'][0]}")
        dp = data_mat['dropout'][0]
        print(np.where(X ==0)[0].shape[0]/(X.shape[0]*X.shape[1]))


        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = utils.normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])
        print(X.shape, count_X.shape)

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        cluster_number = int(max(Y) - min(Y) + 1)
        zeros = np.min(X, axis = 0)
        pxt = PCA(2).fit_transform(X)
        dresults = {
            "dataset": dataset,
            "dropout": data_mat['dropout'][0],
            "nclust": cluster_number,
            "original": utils.evaluate(X, Y, cluster_number)[1],
            "pca": utils.evaluate(pxt, Y, cluster_number)[1]
        }
        # 3 models
        st_results = []
        predictions1 = []
        predictions2 = []
        nm = "500epoch"
        f = []
        for i in range(3):
            r1 = loop.self_train_clustering(X,
                   Y,
                   cluster_number,
                   args,
                   augm_zeros=None,
                   nb_zeros= "random", 
                   random=False,
                   perc=0.1,
                   augm_value=0,
                   model_name = "STClustering",
                   epochs = 500)
            f.append(r1['features'])
            st_results.append(r1)
            predictions1.append(r1['pred_kmeans_representation'])
            predictions2.append(r1['pred'])
            dresults[f"method1_{i}"] = r1["aris_kmeans_representation"][-1]
            dresults[f"method2_{i}"] = r1["aris"][-1]

        ft = np.hstack(f)
        kmeans = KMeans(n_clusters=cluster_number, init="k-means++", random_state=0)
        pred = kmeans.fit_predict(ft)
        predictions1.append(pred)
        predictions2.append(pred)
        dresults[f"method3"] = adjusted_rand_score(Y, pred)

        df = df.append(dresults, ignore_index=True)
        print(dresults)

        all_res = dresults.copy()
        all_res["st"] = st_results

        with open(f"{path}output/{category}/results_{dataset}_{run}.pickle", 'wb') as handle:
            pickle.dump(dresults, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(f"{path}output/{category}/detailed_results_{dataset}_{run}.pickle", 'wb') as handle:
            pickle.dump(all_res, handle, protocol=pickle.HIGHEST_PROTOCOL)

        df.to_pickle(f"{path}output/pickle_results/{category}/{category}_run_{run}.pkl")

In [ ]:
run = 0
df = pd.read_pickle(f"{path}output/pickle_results/{category}/{category}_run_{run}.pkl")
df.head()